In [ ]:
from prettytable import PrettyTable
import sqlite3
import sys

In [27]:
class DB_Manager:
    def __init__(self):
        self.conn = sqlite3.connect(file_path)
        self.cursor = self.conn.cursor()

    def execute(self, sql):
        self.cursor.execute(sql)
        rows = self.cursor.fetchall()
        table = PrettyTable()
        table.field_names = [description[0] for description in self.cursor.description]
        table.add_rows(rows)
        print(table)

    def start(self):
        print("""ГОЛОВНЕ МЕНЮ
            \n Оберіть дію:
            \n1. Створити таблицю.
            \n2. Оглянути та редагувати таблицю.
            \n3. Створити звіт.
            \n4. Завершити роботу.
            """)
        num = int(input("Введіть номер дії: "))

        if num == 1:
            print("СТВОРЮЄМО ТАБЛИЦЮ...")
            print("Для повернення в головне меню введіть МЕНЮ")
            self.create_table()
        elif num == 2:
            print("РЕДАГУЄМО ДАНІ...")
            print("Для повернення в головне меню введіть МЕНЮ")
            self.edit()
        elif num == 3:
            print("РОБИМО ЗВІТ...")
            print("Для повернення в головне меню введіть МЕНЮ")
            self.report()
        elif num == 4:
            print("""Дякуємо за Вашу роботу. Гарного дня! 
                  \n ЗАКРИТТЯ ПРОГРАМИ...""")
            sys.exit()
        else:
            print("Переконайтеся, що Ви ввели правильний номер")
            self.start()

    def create_table(self):
        self.table_name = input("Напишіть назву таблиці: ")
        print(f"Назва таблиці:\n{self.table_name.upper()}")

        sql_check = f"""SELECT name FROM sqlite_master WHERE type='table' AND name='{self.table_name}';"""
        self.cursor.execute(sql_check)
        result = self.cursor.fetchone()
        if result:
            print(f"Таблиця з назвою '{self.table_name}' вже існує. Будь ласка, введіть іншу назву.")
            self.create_table()

        num_col = int(input("Введіть кількість стовпців: "))
        columns = []
        for _ in range(num_col):
            name_col = input("Введіть назву стовпця (у лапках): ")
            type_col = input("Введіть тип даних стовпця (INTEGER/TEXT): ")
            columns.append(f"{name_col} {type_col}")

        columns_str = ', '.join(columns)
        sql = f"""CREATE TABLE "{self.table_name}" ({columns_str})"""
        self.cursor.execute(sql)
        self.conn.commit()

        self.get_table()

        print("""Бажаєте заповнити таблицю?
            \n > введіть 1, щоб продовжити
            \n > введіть 2, щоб повернутись до головного меню""")

        x = int(input())
        if x == 2:
            self.start()
        elif x == 1:
            self.add_row()
            
            
    def get_table(self):
        sql = f"""SELECT * FROM {self.table_name}"""
        self.execute(sql)

    def choose_table(self):
        self.table_name = []
        sql = """SELECT name FROM sqlite_master WHERE type='table';"""
        self.cursor.execute(sql)
        self.edheadlist = self.cursor.fetchall()
        print('Список таблиць:\n', self.edheadlist)

        self.table_name = input("Введіть назву таблиці: ")
        self.get_table()

    def edit(self):
        self.choose_table()
        print("""Оберіть дію:
        \n1. Редагувати дані
        \n2. Видалити рядок
        \n3. Додати рядок
        \n4. Повернутись до головного меню""")
        x = int(input())
        if x == 4:
            self.start()
        elif x == 3:
            self.add_row()
        elif x == 2:
            self.delete_row()
        elif x == 1:
            self.edit_data()

    def add_row(self):
        column = input('\nОберіть колонку:\n')
        data = input('\nВведіть дані:\n')

        sql = f"""INSERT INTO {self.table_name} ({column}) VALUES ('{data}')"""
        self.cursor.execute(sql)
        self.conn.commit()

        self.get_table()

        print("""Бажаєте продовжити додавання даних?
                \n > введіть 1, щоб продовжити
                \n > введіть 2, щоб повернутись до головного меню""")

        x = int(input())
        if x == 2:
            sql = f"""SELECT * FROM {self.table_name}"""
            self.execute(sql)
            self.start()
        elif x == 1:
            self.add_row()

    def delete_row(self):
        row_num = int(input("Введіть номер рядка для видалення: "))
        if row_num < 1 or row_num > len(self.table_name):
            print("Невірний номер рядка. Спробуйте знову.")
            self.delete_row()
            
        sql = self.cursor.execute(
            f"""DELETE FROM {self.table_name} WHERE ROWID = {row_num}""")
        self.conn.commit()
        sql = f"""SELECT * FROM {self.table_name} """
        self.execute(sql)
        
        print("""Бажаєте продовжити видалення даних?
                \n > введіть 1, щоб продовжити
                \n > введіть 2, щоб повернутись до головного меню""")

        x = int(input())
        if x == 1:
            self.delete_row()
        else:
            self.start()

    def edit_data(self):
        self.get_table()
        selected_column = input("Оберіть колонку:\n")
        row_id = int(input("Оберіть рядок:\n"))

        if row_id < 1 or row_id > len(self.table_name):
            print("Невірний номер рядка. Спробуйте знову.")
            self.edit_data()

        new_data = input("Введіть нові дані:\n")
        sql = f"UPDATE {self.table_name} SET {selected_column} = ? WHERE rowid = ?"
        self.cursor.execute(sql, (new_data, row_id))
        self.conn.commit()

        self.get_table()

        print("""Бажаєте продовжити редагування даних?
                \n > введіть 1, щоб продовжити
                \n > введіть 2, щоб повернутись до головного меню""")

        x = int(input())
        if x == 1:
            self.edit_data()
        else:
            self.start()
        
    

    """ 3 - СТВОРЕННЯ ЗВІТУ """
    def report(self):
        self.choose_table()
        name_column = input("Введіть назву стовпця:\n")
        data_report = input("Введіть дані, які хочете знайти:\n")
        sql = f"""SELECT * FROM {self.table_name}
            WHERE {name_column} = '{data_report}'"""
        self.execute(sql)

        print("""Оберіть дію:
                \n1. Відсортувати звіт.
                \n2. Виконати фільтрацію.
                \n3. Друкувати звіт.
                \n4. Повернутись до головного меню.""")
        x = int(input())
        if x == 1:
            self.sort_report()
        elif x == 2:
            self.filter_report()
        elif x == 3:
            self.print_report()
        elif x == 4:
            self.start()
        else:
            print("Переконайтеся, що Ви ввели правильний номер. Спробуйте знову.")
            self.report()

    def sort_report(self):
        column = input("Введіть назву стовпця для сортування:\n")
        sql = f"""SELECT * FROM {self.table_name} ORDER BY {column}"""
        self.execute(sql)
        self.report()

    def filter_report(self):
        column = input("Введіть назву стовпця для фільтрації:\n")
        value = input("Введіть значення для фільтрації:\n")
        print("""Оберіть тип фільтрації:
                \n1. Більше
                \n2. Менше
                \n3. Дорівнює""")
        filter_type = int(input("Оберіть тип фільтрації"))

        if filter_type == 1:
            sql = f"""SELECT * FROM {self.table_name} WHERE {column} > '{value}'"""
        elif filter_type == 2:
            sql = f"""SELECT * FROM {self.table_name} WHERE {column} < '{value}'"""
        elif filter_type == 3:
            sql = f"""SELECT * FROM {self.table_name} WHERE {column} = '{value}'"""
        else:
            print("Переконайтеся, що Ви ввели правильний номер. Спробуйте знову.")
            self.filter_report()

        self.execute(sql)
        self.report()

    def print_report(self):
        name_report = input("Введіть назву для збереження звіту:\n")
        with open(f"{name_report}_data.txt", "w") as f:
            sql = f"""SELECT * FROM {self.table_name}"""
            self.cursor.execute(sql)
            rows = self.cursor.fetchall()
            for row in rows:
                f.write(str(row) + "\n")
        print("Звіт збережено у файлі:", f"{name_report}_data.txt")
        self.report()

program = DB_Manager()
program.start()

ГОЛОВНЕ МЕНЮ
            
Оберіть дію:
            
1. Створити таблицю.
            
2. Оглянути та редагувати таблицю.
            
3. Створити звіт.
            
4. Завершити роботу.
            


ValueError: invalid literal for int() with base 10: ''